### Contents
- [装饰器](#decorator)
    - [函数装饰器](#function_dec)
    - [Class装饰器](#class_dec)
    - [保持装饰前后函数属性一致](#dec_attr)
    - [带参数的装饰器](#dec_para)
    - [双重装饰](#dec_double)

- [装饰器应用](#apply)
    - [函数调用计数](#counter)
    - [闭包-斐波那契数列](#closure_fibonacci)

## <a id='decorator'>装饰器</a>

### <a id='function_dec'>函数装饰器</a>

接收函数作为参数，并返回函数

In [1]:
'''
定义装饰器
'''
def dec(func):
    def wrapper(*args, **kwargs):  #保持与原func的参数一致
        #do something
        ret = func(*args, **kwargs)
        #do something
        return ret   #保证返回的数据结构与原func一致
    print('decorated:', id(wrapper))  #每调用一次dec, 就重新定义一个wrapper
    return wrapper

In [2]:
'''
使用装饰器
'''
@dec     # -> foo = dec(foo) -> wrapper
def foo():  
    pass

foo()   # -> wrapper()

decorated: 4576949520


In [3]:
@dec
def bar(): 
    pass

bar()

decorated: 4576949792


### <a id='class_dec'>Class装饰器</a>
```
装饰器不一定是函数，也可以是class
只是此时经修饰后返回的已经不再是函数，而是类实例
class装饰器实际应用的是类实例的__call__方法
```


In [6]:
class A(object):
    def __init__(self, func):  #在初始化实例时传入function
        print('initiating...')
        self.func = func
    def __call__(self, *args, **kwargs): 
        #do something
        print('calling...')
        ret = self.func(*args, **kwargs)
        #do something
        return ret

@A  # -> foo = A(foo), 此处创建一个A实例
def foo():
    pass

initiating...


In [7]:
type(foo)

__main__.A

In [9]:
foo()  #调用A实例的call方法  instance.__call__()

calling...


### <a id='dec_attr'>保持装饰前后属性一致</a>
经装饰后的函数已经不是原函数，此时函数名、文档、所属模块都不是原函数的

In [22]:
def dec(func):
    def wrapper():
        "hi this is wrapper"
        return func()
    return wrapper

@dec 
def foo():
    "hi this is foo"
    pass

print(foo.__name__, ',', foo.__doc__)

wrapper , hi this is wrapper


> 要保持前后一致可以对wrapper函数再装饰，将原函数的属性赋值给wrapper相应的属性：

In [24]:
from functools import wraps
def dec(func):
    @wraps(func)  #属性转移
    def wrapper():
        "hi this is wrapper"
        return func()
    return wrapper

@dec 
def foo():
    "hi this is foo"
    pass

print(foo.__name__, ',', foo.__doc__)

foo , hi this is foo


### <a id='dec_para'>带参数的装饰器</a>

In [12]:
def weights(w):
    def dec(func):
        def wrapper(*args, **kwargs):
            return w*func(*args, **kwargs)
        return wrapper
    return dec


In [15]:
@weights(2)  #加权2 -> foo = weights(2)(foo)
def foo(x):
    return x

foo(2)

4

In [16]:
@weights(5)  #加权5
def bar(x):
    return x
bar(2)

10

### <a id='dec_double'>双重装饰</a>

In [18]:
def double(name):
    def dec(func):
        print('decorating from', name,  func.__name__)
        def wrapper(*args, **kwargs):
            return func(*args, **kwargs)
        return wrapper
    return dec

@double('B')
@double('A')     #自下而上，从最挨着foo的装饰器开始装饰： -> foo = B(A(foo))
def foo():
    pass

decorating from A foo
decorating from B wrapper



## <a id='apply'>装饰器应用</a>

### <a id='counter'>函数调用计数</a>

In [5]:
'''
通过创建wrapper的函数属性来记录被装饰的
'''
from functools import wraps
def call_count(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        wrapper.count += 1   #跟踪计数该函数的调用
        return func(*args, **kwargs)
    wrapper.count = 0  #初始化刚刚定义函数的属性
    return wrapper

In [6]:
@call_count
def ff():
    pass
ff()
ff()
ff.count

2

In [7]:
@call_count
def bb():
    pass
print(bb.count)    #count属性跟随具体函数，不互相影响
print(ff.count)

0
2


### <a id='closure_fibonacci'>闭包-斐波那契数列</a>

> Fibonacci递归调用次数:
```
1 -> 1
2 -> 1
3 -> 2, 1 -> 1+1+1=3
4 -> 3, 2 -> 3+1+1=5
5 -> 4, 3 -> 5+3+1=9
...
```

In [9]:
'''
考察Fibnacci递归调用次数
'''
def cal_calls(x):
    @call_count    #这么写是为了单独给每个x定义一个新函数， 以便单独统计求第x个数时所调用的总次数
    def fibonacci_recursive(n):
        if n <= 1:
            return 0
        if n == 2:
            return 1
        return fibonacci_recursive(n-1) + fibonacci_recursive(n-2)
    fibonacci_recursive(x)
    print(x, fibonacci_recursive.count)

for i in range(1,11):
    cal_calls(i)

1 1
2 1
3 3
4 5
5 9
6 15
7 25
8 41
9 67
10 109


> 记忆化后的递归调用次数：
```
1 -> 1
2 -> 1
3 -> 2,1 -> 1+1+1=3  本身调用也算一次
4 -> 3,2 -> 3+1=4    跟原来不同，算到2以上时，2本身已经被算3时存下来了
5 -> 4,3 -> 4+1=5   
...
```

In [10]:
'''
记忆：函数参数 -> 函数返回
限制：函数参数可哈希，做字典的键
'''
def memorized(func):
    mems = {}  #闭包, 存储中间结果
    @wraps(func)
    def wrapper(n):
        if n not in mems:
            wrapper.count += 1  #此为一次调用
            mems[n] = func(n)  #只有没算过的才会调用
        return mems[n]
    wrapper.count = 0
    return wrapper


In [11]:
'''
看看如果加了记忆存储后的调用次数会是多少
'''
def cal_call2(x):
    @memorized
    def fibonacci_recursive(n):   #单独为每个x定义一个斐波那契函数
        if n <= 1:
            return 0
        if n == 2:
            return 1
        return fibonacci_recursive(n-1) + fibonacci_recursive(n-2)
    fibonacci_recursive(x)
    print(x, fibonacci_recursive.count)

for i in range(1,11):
    cal_call2(i)

1 1
2 1
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10


> 非递归方式实现斐波那契数列
算法复杂度：
```
1 -> 1
2 -> 1+1 -> 2  a移位1次
3 -> 1+2 -> 3  a移位2次
4 -> 1+3 -> 4  a移位3次
5 -> 1+4 -> 5  a移位5次
...
```

In [12]:
'''
函数中没有引用过去的成果，记忆功能在此不起作用
'''
def fibonacci(n):
    a, b = 0, 1
    if n > 1:
        for i in range(1, n):
            a, b = b, a+b
    return a

In [13]:
[fibonacci(x) for x in range(1, 11)]

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]